In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# read csv
data = pd.read_csv("./../../datasets/All.csv")
# convert dates
data["Hour"] = data["Hour"].astype("str")
data["DateTime"] = pd.to_datetime(data["Date"] + " " + data["Hour"] + ":00:00")
del data["Hour"]
del data["Date"]
# set datetime index
data.index = pd.DatetimeIndex(data.DateTime)
# fix no2 col
data["NO2"] = data["NO2-ug-per-m3"].astype(int)
del data["NO2-ug-per-m3"]
# fix temp col
data = data[data["air_temp_deg_c"] > -500]
data.head()

,Cars-Direction-1,Cars-Direction-2,Cars-Total,Day-Of-Week,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,...,rain_millimeters,did_it_rain,air_temp_deg_c,relative_humidity_percent,e5,e10,diesel,Number of vessels,DateTime,NO2
DateTime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 04:00:00,60,44,104,2,0,1,0,0,0,0,...,0.2,1,7.8,94.0,1.409000,1.389000,1.289000,3,2019-01-01 04:00:00,8
2019-01-01 05:00:00,44,33,77,2,0,1,0,0,0,0,...,0.0,1,8.1,94.0,1.412924,1.387101,1.272671,5,2019-01-01 05:00:00,5
2019-01-01 06:00:00,46,43,89,2,0,1,0,0,0,0,...,0.1,1,8.2,83.0,1.389000,1.365522,1.241174,4,2019-01-01 06:00:00,5
2019-01-01 07:00:00,39,41,80,2,0,1,0,0,0,0,...,0.0,0,7.0,75.0,1.377876,1.353382,1.229562,3,2019-01-01 07:00:00,5
2019-01-01 08:00:00,26,35,61,2,0,1,0,0,0,0,...,0.0,0,7.0,73.0,1.361500,1.336188,1.205406,3,2019-01-01 08:00:00,6


In [14]:
X = data[["Cars-Total", "diesel", "wind_meters_per_second"]].to_numpy()
Y = data["NO2"].to_numpy()
(X.shape, Y.shape)

((93, 3), (93,))

In [15]:
display(data["Cars-Total"].max())
display(data["diesel"].max())
display(data["wind_meters_per_second"].max())
display(data["NO2"].max())

1070

1.2990000000000002

13.0

99

In [16]:
x_scale_factors = [1000.0, 10.0, 10.0]
y_scale_factor = 100.0

X[:, 0] /= x_scale_factors[0]
X[:, 1] /= x_scale_factors[1]
X[:, 2] /= x_scale_factors[2]
Y = Y / y_scale_factor

In [45]:
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import Model

print("TensorFlow version:", tf.__version__)

model = tf.keras.Sequential()
for i in range(2):
    model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation="linear"))

model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error'])
model.fit(X, Y, batch_size=64, epochs=10, validation_split=0.2)

acc = z_metrics.getAccuracy(model, X, Y)
print(len(acc))



TensorFlow version: 2.5.0
Epoch 1/10
2/2 [==============================] - 0s 111ms/step - loss: 0.4172 - mean_absolute_error: 0.5472 - val_loss: 0.3104 - val_mean_absolute_error: 0.5309
Epoch 2/10
2/2 [==============================] - 0s 17ms/step - loss: 0.4043 - mean_absolute_error: 0.5371 - val_loss: 0.3010 - val_mean_absolute_error: 0.5222
Epoch 3/10
2/2 [==============================] - 0s 21ms/step - loss: 0.3927 - mean_absolute_error: 0.5276 - val_loss: 0.2918 - val_mean_absolute_error: 0.5133
Epoch 4/10
2/2 [==============================] - 0s 22ms/step - loss: 0.3811 - mean_absolute_error: 0.5185 - val_loss: 0.2827 - val_mean_absolute_error: 0.5045
Epoch 5/10
2/2 [==============================] - 0s 23ms/step - loss: 0.3698 - mean_absolute_error: 0.5095 - val_loss: 0.2737 - val_mean_absolute_error: 0.4956
Epoch 6/10
2/2 [==============================] - 0s 18ms/step - loss: 0.3594 - mean_absolute_error: 0.5008 - val_loss: 0.2651 - val_mean_absolute_error: 0.4871
Epoch 7